<a href="https://colab.research.google.com/github/jlbs86/cutonala_24B_parallel_programming/blob/HAGC_Task_mpi4py_library/mpi4py_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install mpi4py

  Using cached mpi4py-4.0.0.tar.gz (464 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.0-cp310-cp310-linux_x86_64.whl size=4266267 sha256=a7d2ce330835e56f49223f94d3149cae261aa152a653d48fe3ad01b7105c7c71
  Stored in directory: /root/.cache/pip/wheels/96/17/12/83db63ee0ae5c4b040ee87f2e5c813aea4728b55ec6a37317c
Successfully built mpi4py


In [31]:
! mpiexec --oversubscribe --allow-run-as-root -n 4 python /content/sample_data/example.py

Rank 0 received data: {'a': 7, 'b': 3.14}
Rank 1 received data: {'a': 7, 'b': 3.14}
